In [1]:
import pandas as pd
import numpy as np
import math
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import re
import gc

import warnings
warnings.filterwarnings("ignore")
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter

# Import libraries
import nltk.translate.chrf_score as chrf
from rouge import Rouge
import nltk
import nltk.translate.gleu_score as gleu

import numpy
import os

try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')
import nltk.translate.meteor_score as meteor
import nltk.translate.nist_score as nist
import nltk.translate.bleu_score as bleu

#import spacy
from nltk.tokenize import RegexpTokenizer

from sklearn.metrics.pairwise import cosine_similarity
import itertools

import codecs
import jieba

#Import nltk Snowball stemmer
from nltk.stem.snowball import SnowballStemmer



In [2]:
try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')

In [3]:
scores_cs = pd.read_csv("corpus/cs-en/scores.csv")
scores_de = pd.read_csv("corpus/de-en/scores.csv")
scores_en_fi = pd.read_csv("corpus/en-fi/scores.csv")
scores_en_zh = pd.read_csv("corpus/en-zh/scores.csv")
scores_ru = pd.read_csv("corpus/ru-en/scores.csv")
scores_zh = pd.read_csv("corpus/zh-en/scores.csv")

In [3]:
# DEPRECATED - WAS LEFT HERE TO SHOW HOW CONTRACTIONS LOOK LIKE

# contractions = { 
# "ain't": "am not / are not / is not / has not / have not",
# "aren't": "are not / am not",
# "can't": "cannot",
# "can't've": "cannot have",
# "'cause": "because",
# "could've": "could have",
# "couldn't": "could not",
# "couldn't've": "could not have",
# "didn't": "did not",
# "doesn't": "does not",
# "don't": "do not",
# "hadn't": "had not",
# "hadn't've": "had not have",
# "hasn't": "has not",
# "haven't": "have not",
# "he'd": "he had / he would",
# "he'd've": "he would have",
# "he'll": "he shall / he will",
# "he'll've": "he shall have / he will have",
# "he's": "he has / he is",
# "how'd": "how did",
# "how'd'y": "how do you",
# "how'll": "how will",
# "how's": "how has / how is / how does",
# "I'd": "I had / I would",
# "I'd've": "I would have",
# "I'll": "I shall / I will",
# "I'll've": "I shall have / I will have",
# "I'm": "I am",
# "I've": "I have",
# "isn't": "is not",
# "it'd": "it had / it would",
# "it'd've": "it would have",
# "it'll": "it shall / it will",
# "it'll've": "it shall have / it will have",
# "it's": "it has / it is",
# "let's": "let us",
# "ma'am": "madam",
# "mayn't": "may not",
# "might've": "might have",
# "mightn't": "might not",
# "mightn't've": "might not have",
# "must've": "must have",
# "mustn't": "must not",
# "mustn't've": "must not have",
# "needn't": "need not",
# "needn't've": "need not have",
# "o'clock": "of the clock",
# "oughtn't": "ought not",
# "oughtn't've": "ought not have",
# "shan't": "shall not",
# "sha'n't": "shall not",
# "shan't've": "shall not have",
# "she'd": "she had / she would",
# "she'd've": "she would have",
# "she'll": "she shall / she will",
# "she'll've": "she shall have / she will have",
# "she's": "she has / she is",
# "should've": "should have",
# "shouldn't": "should not",
# "shouldn't've": "should not have",
# "so've": "so have",
# "so's": "so as / so is",
# "that'd": "that would / that had",
# "that'd've": "that would have",
# "that's": "that has / that is",
# "there'd": "there had / there would",
# "there'd've": "there would have",
# "there's": "there has / there is",
# "they'd": "they had / they would",
# "they'd've": "they would have",
# "they'll": "they shall / they will",
# "they'll've": "they shall have / they will have",
# "they're": "they are",
# "they've": "they have",
# "to've": "to have",
# "wasn't": "was not",
# "we'd": "we had / we would",
# "we'd've": "we would have",
# "we'll": "we will",
# "we'll've": "we will have",
# "we're": "we are",
# "we've": "we have",
# "weren't": "were not",
# "what'll": "what shall / what will",
# "what'll've": "what shall have / what will have",
# "what're": "what are",
# "what's": "what has / what is",
# "what've": "what have",
# "when's": "when has / when is",
# "when've": "when have",
# "where'd": "where did",
# "where's": "where has / where is",
# "where've": "where have",
# "who'll": "who shall / who will",
# "who'll've": "who shall have / who will have",
# "who's": "who has / who is",
# "who've": "who have",
# "why's": "why has / why is",
# "why've": "why have",
# "will've": "will have",
# "won't": "will not",
# "won't've": "will not have",
# "would've": "would have",
# "wouldn't": "would not",
# "wouldn't've": "would not have",
# "y'all": "you all",
# "y'all'd": "you all would",
# "y'all'd've": "you all would have",
# "y'all're": "you all are",
# "y'all've": "you all have",
# "you'd": "you had / you would",
# "you'd've": "you would have",
# "you'll": "you shall / you will",
# "you'll've": "you shall have / you will have",
# "you're": "you are",
# "you've": "you have"
# }

# contractions_re = re.compile('(%s)' % '|'.join(contractions.keys()))
# def expand_contractions(s, contractions = contractions):
#     def replace(match):
#         return contractions[match.group(0)]
#     return contractions_re.sub(replace, s)

# expand_contractions("ain't stop")


In [4]:
# Work with contractions as previous cell but pycontractes lib chooses a proper form
# instead ambiguous "you'd" -> "you had / you would"

from pycontractions import Contractions
cont = Contractions('GoogleNews-vectors-negative300.bin')

# prevents loading on first expand_texts call
# usually takes time
cont.load_models()

In [5]:
a = scores_de.translation
a[6]

"It's now time to look at the needs of community legal centres and their client communities, and for the Palaszczuk government to invest in this important work."

In [6]:
b = pd.Series(list(cont.expand_texts(a, precise=True)))

In [7]:
# Check how contracion expanding worked
b[6]

'it is now time to look at the needs of community legal centres and their client communities, and for the Palaszczuk government to invest in this important work.'

In [8]:
# As contraction extractions work really slow
# they were done outside preprocessing function
references_cs = pd.Series(list(cont.expand_texts(scores_cs['reference'], precise=True)))
references_de = pd.Series(list(cont.expand_texts(scores_de['reference'], precise=True)))
references_ru = pd.Series(list(cont.expand_texts(scores_ru['reference'], precise=True)))
references_zh = pd.Series(list(cont.expand_texts(scores_zh['reference'], precise=True)))

In [9]:
# Same for translations
translation_cs = pd.Series(list(cont.expand_texts(scores_cs['translation'], precise=True)))
translation_de = pd.Series(list(cont.expand_texts(scores_de['translation'], precise=True)))
translation_ru = pd.Series(list(cont.expand_texts(scores_ru['translation'], precise=True)))
translation_zh = pd.Series(list(cont.expand_texts(scores_zh['translation'], precise=True)))

In [10]:
# Check how expanding contractions worked
print(scores_de['translation'][6], '   ---->  \n',translation_de[6])

It's now time to look at the needs of community legal centres and their client communities, and for the Palaszczuk government to invest in this important work.    ---->  
 it is now time to look at the needs of community legal centres and their client communities, and for the Palaszczuk government to invest in this important work.


In [11]:
# release references and 'garbage collect' to free memory
import gc
del cont
gc.collect()

98

In [12]:
def preprocessing(series, pontuation = False, zh = False):
    processed_corpus = []
    #stop_words = set(stopwords.words())
    for i in tqdm(range(len(series))):
        text = series[i]
        
        #remove tags
        text = BeautifulSoup(text).get_text()
        
        #Remove punctuations
        if pontuation==True:
            text = re.sub(r'[\s+\.\!\/_,$%^*(+\"\']+|[+——！，？、~@#￥%……&*（）:：；《）《》“”()»〔〕-]+', ' ', text)
        else:
            pass

        #Convert to lowercase
        text = text.lower()
        
        # Convert to list from string
        
        # For en-zh language pair use jieba for words cut
        if zh == True:
            text = jieba.cut_for_search(text)
        else:
            text = text.split()

        #Lemmatisation
        lem = WordNetLemmatizer()
        #text = [lem.lemmatize(word) for word in text if not word in stop_words] 
        text = " ".join(text)
        processed_corpus.append(text)
    return processed_corpus

In [13]:
references_cs = preprocessing(references_cs)
references_de = preprocessing(references_de)
references_en_fi = preprocessing(scores_en_fi['reference'])
references_en_zh = preprocessing(scores_en_zh['reference'], zh = True)
references_ru = preprocessing(references_ru)
references_zh = preprocessing(references_zh)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\murmu\AppData\Local\Temp\jieba.cache
Loading model cost 0.712 seconds.
Prefix dict has been built successfully.


In [14]:
translation_cs = preprocessing(translation_cs)
translation_de = preprocessing(translation_de)
translation_en_fi = preprocessing(scores_en_fi['translation'])
translation_en_zh = preprocessing(scores_en_zh['translation'], zh = True)
translation_ru = preprocessing(translation_ru)
translation_zh = preprocessing(translation_zh)

In [15]:
# METRICS from your file text_mining1

In [16]:
def chrf_metric(translation, references):
    chrf_metric = []
    for i in range(len(translation)):
        row = chrf.sentence_chrf([translation[i]], references[i])
        chrf_metric.append(row)
    return chrf_metric

chf_cs = chrf_metric(translation_cs, references_cs)
scores_cs['chf'] = chf_cs
chf_de = chrf_metric(translation_de, references_de)
scores_de['chf'] = chf_de
chf_en_fi = chrf_metric(translation_en_fi, references_en_fi)
scores_en_fi['chf'] = chf_en_fi
chf_en_zh = chrf_metric(translation_en_zh, references_en_zh)
scores_en_zh['chf'] = chf_en_zh
chf_ru = chrf_metric(translation_ru, references_ru)
scores_ru['chf'] = chf_ru
chf_zh = chrf_metric(translation_zh, references_zh)
scores_zh['chf'] = chf_zh

In [17]:
def gleu_metric(translation, references):
    gleu_metric = []
    for i in range(len(translation)):
        row = gleu.sentence_gleu([str(references[i]).split()], str(translation[i]).split(), min_len=1, max_len=2)
        gleu_metric.append(row)
    return gleu_metric

In [18]:
gleu_cs = gleu_metric(translation_cs, references_cs)
scores_cs['gleu'] = gleu_cs

gleu_de = gleu_metric(translation_de, references_de)
scores_de['gleu'] = gleu_de

gleu_en_fi = gleu_metric(translation_en_fi, references_en_fi)
scores_en_fi['gleu'] = gleu_en_fi

gleu_en_zh = gleu_metric(translation_en_zh, references_en_zh)
scores_en_zh['gleu'] = gleu_en_zh

gleu_ru = gleu_metric(translation_ru, references_ru)
scores_ru['gleu'] = gleu_ru

gleu_zh = gleu_metric(translation_zh, references_zh)
scores_zh['gleu'] = gleu_zh

In [19]:
def meteor_metric(translation, references):
    meteor_metric = []
    for i in range(len(translation)):
        row = (meteor.meteor_score([translation[i]], references[i]))
        meteor_metric.append(row)
    return meteor_metric

In [20]:
meteor_cs = meteor_metric(translation_cs, references_cs)
scores_cs['meteor'] = meteor_cs

meteor_de = meteor_metric(translation_de, references_de)
scores_de['meteor'] = meteor_de

meteor_en_fi = meteor_metric(translation_en_fi, references_en_fi)
scores_en_fi['meteor'] = meteor_en_fi

meteor_en_zh = meteor_metric(translation_en_zh, references_en_zh)
scores_en_zh['meteor'] = meteor_en_zh

meteor_ru = meteor_metric(translation_ru, references_ru)
scores_ru['meteor'] = meteor_ru

meteor_zh = meteor_metric(translation_zh, references_zh)
scores_zh['meteor'] = meteor_zh

In [21]:
def bleu_metric(translation, references):
    bleu_metric = []
    for i in range(len(translation)):
        row = (bleu.sentence_bleu([translation[i]], references[i]))
        bleu_metric.append(row)
    return bleu_metric

bleu_cs = bleu_metric(translation_cs, references_cs)
scores_cs['bleu'] = bleu_cs

bleu_de = bleu_metric(translation_de, references_de)
scores_de['bleu'] = bleu_de

bleu_en_fi = bleu_metric(translation_en_fi, references_en_fi)
scores_en_fi['bleu'] = bleu_en_fi

bleu_en_zh = bleu_metric(translation_en_zh, references_en_zh)
scores_en_zh['bleu'] = bleu_en_zh

bleu_ru = bleu_metric(translation_ru, references_ru)
scores_ru['bleu'] = bleu_ru

bleu_zh = bleu_metric(translation_zh, references_zh)
scores_zh['bleu'] = bleu_zh

In [22]:

def nist_metric(translation, references):
    nist_metric = []
    for i in range(len(translation)):
        row = (nist.sentence_nist([translation[i]], references[i],n=1))
        nist_metric.append(row)
    return nist_metric

In [23]:
nist_cs = nist_metric(translation_cs, references_cs)
scores_cs['nist'] = nist_cs

nist_de = nist_metric(translation_de, references_de)
scores_de['nist'] = nist_de

nist_en_fi = nist_metric(translation_en_fi, references_en_fi)
scores_en_fi['nist'] = nist_en_fi

nist_en_zh = nist_metric(translation_en_zh, references_en_zh)
scores_en_zh['nist'] = nist_en_zh

nist_ru = nist_metric(translation_ru, references_ru)
scores_ru['nist'] = nist_ru

nist_zh = nist_metric(translation_zh, references_zh)
scores_zh['nist'] = nist_zh

In [24]:
def corr(scores):
    chf = scores['z-score'].corr(scores['chf'])
    gleu = scores['z-score'].corr(scores['gleu'])
    meteor = scores['z-score'].corr(scores['meteor'])
    bleu = scores['z-score'].corr(scores['bleu'])
    nist = scores['z-score'].corr(scores['nist'])
    #ribes = scores['z-score'].corr(scores['ribes'])
    corr = [chf, gleu, meteor, bleu, nist]
    return corr

In [25]:
corr_cs = corr(scores_cs)
corr_de = corr(scores_de)
corr_en_fi = corr(scores_en_fi)
corr_en_zh = corr(scores_en_zh)
corr_ru = corr(scores_ru)
corr_zh = corr(scores_zh)

In [26]:
columns = ['chf', 'gleu', 'meteor', 'bleu', 'nist']
data = []
index = ['scores_cs', 'scores_de', 'scores_en_fi', 'scores_en_zh', 'scores_ru', 'scores_zh']
correlation = pd.DataFrame(data, columns=columns, index = index)
correlation.iloc[0] = corr_cs
correlation.iloc[1] = corr_de
correlation.iloc[2] = corr_en_fi
correlation.iloc[3] = corr_en_zh
correlation.iloc[4] = corr_ru
correlation.iloc[5] = corr_zh

In [27]:
# Baseline correlation
correlation

,chf,gleu,meteor,bleu,nist
scores_cs,0.46223,0.427899,0.439972,0.46878,0.334652
scores_de,0.341142,0.310098,0.308036,0.346757,0.227363
scores_en_fi,0.611567,0.494636,0.491475,0.619928,0.423513
scores_en_zh,0.423398,0.449157,0.453092,0.468428,0.432876
scores_ru,0.361416,0.333536,0.336737,0.367581,0.26207
scores_zh,0.341207,0.317924,0.326399,0.351904,0.244659


## Calculate cosine similarities for sentences using word embeddings
 - we use pre-trained word embeddings

In [28]:
# english stopwords
english_stop_words = set(stopwords.words('english')).union(STOP_WORDS)

# chinese stopwords
stopwords_zh = codecs.open('stopwords-zh.txt', 'r', 'utf-8').read().split(',')

# finnish stopwords
stopwords_fi = codecs.open('stop_words_finnish.txt', 'r').read().split(',')

In [29]:
# delete stopwords, lemmatize if needed

def words_count(text):
    tokenizer = RegexpTokenizer(r'\w+')
    # tokenize text
    tokens = tokenizer.tokenize(text)
   
    # delete stop words
    tokens = list(filter(lambda x: x not in english_stop_words, tokens))
    
    # lemmatize the words WordNetLemmatizer from nlkt
    # this made Pearson correlation lower - so was commented
    
    # tokens = list(map(lambda x: nltk.WordNetLemmatizer().lemmatize(x), tokens))
    
    # With spacy lemmatizer (worked better than nltk)
    # it is commented because it made correlation lower
    # tokens = list(map(lambda x: [token.lemma_ for token in nlp(x)], tokens))
    # tokens = list(itertools.chain(*tokens))
    
    return list(Counter(tokens).most_common())

In [30]:
# delete stopwords, lemmatize if needed

def words_count_zh(text):
#     # Tokenize
    tokenizer = RegexpTokenizer(r'\w+')
    # tokenize text
    tokens = tokenizer.tokenize(text)
    # delete stop words
    tokens = list(filter(lambda x: x not in stopwords_zh, tokens))
    return list(Counter(tokens).most_common())

In [56]:
# delete stopwords, lemmatize if needed
def words_count_fi(text):
#   # Tokenize
    tokenizer = RegexpTokenizer(r'\w+')
    # tokenize text
    tokens = tokenizer.tokenize(text)
    
    # delete stop words
    tokens = list(filter(lambda x: x not in stopwords_fi, tokens))

    return list(Counter(tokens).most_common())

In [33]:
# Load the pre-trained GloVe (Global Vectors for Word Representation) word vectors.
# For cs, de, ru, zh that translated to english
# 6B tokens, 400K vocab, (300d version).
# Download file from https://www.kaggle.com/thanakomsn/glove6b300dtxt
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
    return word_to_vec_map
# Download glove.6B.300d.txt to a folder with this jupiter notebook
word_to_vec_map = read_glove_vecs('glove.6B.300d.txt')

In [34]:
# Chinese word vector
# Download word vectors file from https://www.kaggle.com/kerneler/starter-sgns-merge-word-18e5b7b5-9/execution?select=sgns.merge.word
word_to_vec_zh = read_glove_vecs('sgns.merge.word')

In [35]:
# Finnish word vectors
# Download from https://github.com/jmyrberg/finnish-word-embeddings
word_to_vec_fi = read_glove_vecs('cc.fi.300.vec')

In [60]:
# Create column for calculating cosine similarity
scores_cs['Cosine_similarity'], scores_de['Cosine_similarity'] = 0, 0
scores_ru['Cosine_similarity'], scores_zh['Cosine_similarity'] = 0, 0
scores_en_fi['Cosine_similarity'], scores_en_zh['Cosine_similarity'] = 0, 0

In [61]:
# Create lists with languages to work with
ref_list = [references_cs, references_de, references_ru, references_zh, references_en_fi, references_en_zh]
trans_list = [translation_cs, translation_de, translation_ru, translation_zh, translation_en_fi,  translation_en_zh]
scores = [scores_cs, scores_de, scores_ru, scores_zh, scores_en_fi, scores_en_zh]

In [62]:
# Check cosine similarity between two sentences (reference and translation)
# Use different pre-trained word embeddings:
# GloVe - for cs, de, ru, zh that translated to english
# 'sgns.merge.word' - for en-zh pair
# 'cc.fi.300.vec' - for en-fi pair

for doc in range(len(scores)):
    ref_embeddings = np.zeros((len(scores[doc]),300))
    tr_embeddings = np.zeros((len(scores[doc]),300))
    
    norms_doc_embeddings_tr = np.zeros((len(trans_list[doc]),1))
    norms_doc_embeddings_ref = np.zeros((len(ref_list[doc]),1))
    
    Cosine_similarity = []
    
    for i in tqdm(range(len(trans_list[doc]))):
        if doc == 5:
            words_freq = words_count_zh(trans_list[doc][i])
            doc_embeddings = np.zeros(word_to_vec_zh["a"].shape)
        elif doc == 4:
            words_freq = words_count_fi(trans_list[doc][i])
            doc_embeddings = np.zeros(word_to_vec_fi["a"].shape)
        else:
            words_freq = words_count(trans_list[doc][i])
            doc_embeddings = np.zeros(word_to_vec_map["a"].shape)
        num_words = 0
        for word_freq in words_freq:
            word = word_freq[0]
            freq = word_freq[1]
            try:
                #adding word embeddings for each word in the document
                if doc == 5:
                    doc_embeddings += (word_to_vec_zh[word] * freq)
                elif doc == 4:
                    doc_embeddings += (word_to_vec_fi[word] * freq)
                else:
                    doc_embeddings += (word_to_vec_map[word] * freq)
                num_words += freq
            except:
                continue
        try:
            # doing average
            doc_embeddings /= num_words
        except:
            print("divide by zero encountered for article at index "+str(i))
            continue
        norms_doc_embeddings_tr[i,:] = np.sqrt(np.dot(doc_embeddings,doc_embeddings))
        tr_embeddings[i,:] = doc_embeddings
        
        
        if doc == 5:
            words_freq = words_count_zh(ref_list[doc][i])
            doc_embeddings = np.zeros(word_to_vec_zh["a"].shape)
        elif doc == 4:
            words_freq = words_count_fi(trans_list[doc][i])
            doc_embeddings = np.zeros(word_to_vec_fi["a"].shape)

        else:
            words_freq = words_count(ref_list[doc][i])
            doc_embeddings = np.zeros(word_to_vec_map["a"].shape)
        num_words = 0
        for word_freq in words_freq:
            word = word_freq[0]
            freq = word_freq[1]
            try:
                #adding word embeddings for each word in the document
                if doc == 5:
                    doc_embeddings += (word_to_vec_zh[word] * freq)
                elif doc == 4:
                    doc_embeddings += (word_to_vec_fi[word] * freq)
                else:
                    doc_embeddings += (word_to_vec_map[word] * freq)
                num_words += freq
            except:
                continue
        try:
            # doing average
            doc_embeddings /= num_words
        except:
            print("divide by zero encountered for article at index "+str(i))
            continue
        norms_doc_embeddings_ref[i,:] = np.sqrt(np.dot(doc_embeddings,doc_embeddings))
        ref_embeddings[i,:] = doc_embeddings
    
        # Calculate cosine similarity using dot product of two vectors
        # and norms of vectors
        
        #  The cosine similarity depends on the angle between  vectors .
        #  If  vectors  are very similar, their cosine similarity will be close to 1
        norm_prods = norms_doc_embeddings_ref[i] * norms_doc_embeddings_tr[i]
        dot_prod = np.dot(ref_embeddings[i], tr_embeddings[i].reshape(300,1))
        cos_similarity = dot_prod / norm_prods
        
        Cosine_similarity.append(float(cos_similarity))
    
    scores[doc]['Cosine_similarity'] = pd.Series(Cosine_similarity)

In [63]:
def corr_new(scores):
    chf = scores['z-score'].corr(scores['chf'])
    gleu = scores['z-score'].corr(scores['gleu'])
    meteor = scores['z-score'].corr(scores['meteor'])
    bleu = scores['z-score'].corr(scores['bleu'])
    nist = scores['z-score'].corr(scores['nist'])
    cos = scores['z-score'].corr(scores['Cosine_similarity'])
    corr = [chf, gleu, meteor, bleu, nist, cos]
    return corr

In [64]:
corr_cs_new = corr_new(scores_cs)
corr_de_new = corr_new(scores_de)
corr_ru_new = corr_new(scores_ru)
corr_zh_new = corr_new(scores_zh)
corr_en_fi_new = corr_new(scores_en_fi)
corr_en_zh_new = corr_new(scores_en_zh)

In [65]:
columns = ['chf', 'gleu', 'meteor', 'bleu', 'nist', 'Cos_sim']
data = []
index = ['scores_cs', 'scores_de', 'scores_en_fi', 'scores_en_zh', 'scores_ru', 'scores_zh']
correlation = pd.DataFrame(data, columns=columns, index = index)
correlation.iloc[0] = corr_cs_new
correlation.iloc[1] = corr_de_new


correlation.iloc[2] = corr_en_fi_new
correlation.iloc[3] = corr_en_zh_new

correlation.iloc[4] = corr_ru_new
correlation.iloc[5] = corr_zh_new



In [66]:
# Calculate correlation between z-score and cosine_similarity columns
# as well
correlation

,chf,gleu,meteor,bleu,nist,Cos_sim
scores_cs,0.46223,0.427899,0.439972,0.46878,0.334652,0.368188
scores_de,0.341142,0.310098,0.308036,0.346757,0.227363,0.290548
scores_en_fi,0.611567,0.494636,0.491475,0.619928,0.423513,-0.00617554
scores_en_zh,0.423398,0.449157,0.453092,0.468428,0.432876,0.360268
scores_ru,0.361416,0.333536,0.336737,0.367581,0.26207,0.304984
scores_zh,0.341207,0.317924,0.326399,0.351904,0.244659,0.27196


In [85]:
from libvoikko import Voikko
Voikko.setLibrarySearchPath("c:\Voikko")

In [87]:
#Import the Voikko library
import libvoikko

#Define a Voikko class for Finnish
v = libvoikko.Voikko(u"fi")

#A word that might or might not be in base form
#Finnish word "kissoja" means "cats" in English
word = "kissoja"

#Analyze the word
voikko_dict = v.analyze(word)

#Extract the base form as
#analyze() function returns various info for the word
word_baseform = voikko_dict[0]['BASEFORM']

#Print the base form of the word
#This should print "kissa", which is "cat" in English
print(word_baseform)

VoikkoException: Initialization of Voikko failed: No valid dictionaries were found

In [88]:
import sys

from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel

import sparknlp
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from sparknlp.pretrained import ResourceDownloader

from pathlib import Path

if sys.version_info[0] < 3:
    from urllib import urlretrieve
else:
    from urllib.request import urlretrieve

ModuleNotFoundError: No module named 'pyspark'

In [90]:
from uralicNLP import uralicApi
uralicApi.supported_languages()


ModuleNotFoundError: No module named 'uralicNLP'